# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


## Set up initial environment
The cell below instantiates the environment and sets some initial variables:

- brain_name
- action_size: the number of actions that can be performed in the environment
- state_size: the number of values retured from the envionment to represent the current state

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [3]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
counter = 0
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    counter += 1
    #print('dones: {}'.format(dones))
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
print('max episode length: {}'.format(counter))


Total score (averaged over agents) this episode: 0.12299999725073577
max episode length: 1001


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [52]:
# define netowrks
import torch
import torch.nn.functional as F
import torch.optim as optim
import copy

# actor - take in a state and output a distribution of actions
class ActorModel(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(ActorModel, self).__init__()
        self.state_size   = state_size
        self.action_size = action_size

        self.fc1 = torch.nn.Linear(state_size, 256)
        self.fc2 = torch.nn.Linear(256, 256)
        self.out = torch.nn.Linear(256, action_size)

    def forward(self, states):
        batch_size = states.size(0)
        x = F.relu(self.fc1(states))
        x = F.relu(self.fc2(x))
        x = F.tanh(self.out(x))
        return x

# critic - take in a state AND actions - outputs a state value function - V
class CriticModel(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(CriticModel, self).__init__()
        self.state_size   = state_size
        self.action_size = action_size

        self.fc1 = torch.nn.Linear(state_size, 256)
        self.fc2 = torch.nn.Linear(256+action_size, 256)
        self.out = torch.nn.Linear(256, 1)

    def forward(self, states, actions):
        batch_size = states.size(0)
        xs = F.relu(self.fc1(states))
        x = torch.cat((xs, actions), dim=1) #add in actions to the network
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x
    
class StepInfo:
    def __init__(self, step_number, states, actions, rewards, next_states):
        self.step_number = step_number
        self.states = states
        self.actions = actions
        self.rewards = rewards
        self.next_states = next_states
        
    def __str__(self):
        return "step_number: {},  states: {},  actions: {},  rewards: {},  next_states: {}".format(self.step_number, self.states, self.actions, self.rewards, self.next_states)

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state
    
def play_game(env, brain_name, replay_buffer, actor_model, add_noise=True):
    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations     
    # get the current state (for each agent)
    random_seed = 1
    noise = OUNoise(actor_model.action_size, random_seed)

    #Play a game. Add to the replay_buffer
    step_number = 0
    total_rewards = 0.0
    while True:
        state_tensor = torch.from_numpy(states).float().cuda()

        #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions_tensor = actor_model_local(state_tensor)
        actions_np = actions_tensor.detach().cpu().numpy()
        #print("actions: {}".format(actions_np))
        #actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions_np)[brain_name]         # send all actions to tne environment
        next_states = env_info.vector_observations          # get next state (for each agent)
        rewards = env_info.rewards                          # get reward (for each agent)
        total_rewards += np.sum(rewards)
        dones = env_info.local_done                         # see if episode finished

        if add_noise:
            actions_np += noise.sample()
            np.clip(actions_np, -1, 1)
    
        this_step_info = StepInfo(step_number, states, actions_np, rewards, next_states)
        replay_buffer.append(this_step_info)
        step_number += 1

        states = next_states

        if np.any(dones):                                  # exit loop if episode finished
            break
    return total_rewards
            
def soft_update_target(local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
        target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [53]:
# training process (A3C): 
# Feed current_state into actor network to get the action to take in that state
#     Get next_state and rewards from the env

# Feed next_state (s-prime) into critic network to get critic_next_state_reward
#     Train the critic for current state using: current_reward + gamma * critic_next_state_reward

# Calculate Advantage (A) given current_state and action:
#     A(current_state, action) = reward + gamma(critic_next_state_reward) - critic_current_state_reward

# Train the Actor using the Calculated Advantage (A) as a baseline

# N-step bootstrapping???  Lambda (Generalized Advantage Estimate)???
# no replay buffer in AC3
# ************** not sure A3C will work with continuous action spaces

# training process (DDPG) - continuous action spaces
# Query the actor network to get believed best action (BBA)
# The critic evaluates the actor's BBA Given state AND actor's action values as input.
# DDPG uses a replay buffer
# Need a regular and target network for the actor and critic networks.
#  update targets using soft update strategy - e.g. slowly mix in updates.
#  target is used for predictions.

In [54]:
from collections import deque

# instantiate objects that will can be re-used
replay_buffer = deque(maxlen=1001)

actor_model_local   = ActorModel(state_size, action_size).cuda()
actor_model_target  = ActorModel(state_size, action_size).cuda()
critic_model_local  = CriticModel(state_size, action_size).cuda()
critic_model_target = CriticModel(state_size, action_size).cuda()

lr = .001
actor_optimizer = optim.Adam(actor_model_local.parameters(), lr=lr)
critic_optimizer = optim.Adam(critic_model_local.parameters(), lr=lr)




In [55]:
import random
# train the model

epochs = 5
games_per_epoch = 5
learning_iterations_per_game = 1000
gamma = 0.99
tau = 0.01

for epoch in range(epochs):
    
    for game in range(games_per_epoch):
        #play a game
        total_actor_loss = 0.0
        total_critic_loss = 0.0
        total_rewards = play_game(env, brain_name, replay_buffer, actor_model_local)
        
        #do some learning
        for learning_iteration in range(learning_iterations_per_game):
            rand_frame_index = random.randint(0,len(replay_buffer)-1)
            this_replay_buffer = replay_buffer[rand_frame_index]
            
            # ---------------------------- update critic ---------------------------- #
            # Get predicted next-state actions and Q values from target models
            next_states = this_replay_buffer.next_states
            next_states_tensor = torch.from_numpy(next_states).float().cuda()
            actions_next = actor_model_target(next_states_tensor)
            Q_targets_next = critic_model_target(next_states_tensor, actions_next)
            #print("Q_targets_next: {}".format(Q_targets_next))
            
            # Compute Q targets for current states (y_i)
            rewards = np.asarray(this_replay_buffer.rewards)
            rewards_tensor = torch.from_numpy(rewards).float().cuda()
            Q_targets = rewards_tensor + (gamma * Q_targets_next)
            #print("Q_targets: {}".format(Q_targets))
                
            # Compute critic loss
            states = this_replay_buffer.states
            states_tensor = torch.from_numpy(states).float().cuda()
            actions = this_replay_buffer.actions
            actions_tensor = torch.from_numpy(actions).float().cuda()
            Q_expected = critic_model_local(states_tensor, actions_tensor)
            #print("Q_expected: {}".format(Q_expected))
            critic_loss = F.mse_loss(Q_expected, Q_targets)
            
            # Minimize the critic loss
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
            total_critic_loss += critic_loss.item()

            # ---------------------------- update actor ---------------------------- #
            # Compute actor loss
            actions_pred = actor_model_local(states_tensor)
            actor_loss = -critic_model_local(states_tensor, actions_pred).mean()

            # Minimize the loss
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            total_actor_loss += actor_loss.item()

        print("epoch: {} - total_rewards: {:.3f}, critic_loss: {:.5f}, actor_loss: {:.3f}".format(epoch, total_rewards, total_critic_loss/learning_iterations_per_game, total_actor_loss/learning_iterations_per_game))
        # ----------------------- update target networks ----------------------- #
        soft_update_target(critic_model_local, critic_model_target, tau)
        soft_update_target(actor_model_local, actor_model_target, tau)
        
                

epoch: 0 - total_rewards: 10.300, critic_loss: 0.00715, actor_loss: 0.240
epoch: 0 - total_rewards: 10.380, critic_loss: 0.00618, actor_loss: 0.257
epoch: 0 - total_rewards: 0.500, critic_loss: 0.00339, actor_loss: 0.400
epoch: 0 - total_rewards: 1.850, critic_loss: 0.00293, actor_loss: 0.369
epoch: 0 - total_rewards: 1.130, critic_loss: 0.00189, actor_loss: 0.317
epoch: 1 - total_rewards: 1.960, critic_loss: 0.00176, actor_loss: 0.196
epoch: 1 - total_rewards: 16.220, critic_loss: 0.00424, actor_loss: 0.109
epoch: 1 - total_rewards: 15.330, critic_loss: 0.00303, actor_loss: 0.127
epoch: 1 - total_rewards: 15.140, critic_loss: 0.00203, actor_loss: 0.101
epoch: 1 - total_rewards: 17.150, critic_loss: 0.00179, actor_loss: 0.121
epoch: 2 - total_rewards: 20.060, critic_loss: 0.00161, actor_loss: 0.077
epoch: 2 - total_rewards: 17.940, critic_loss: 0.00121, actor_loss: 0.108
epoch: 2 - total_rewards: 8.020, critic_loss: 0.00120, actor_loss: 0.095
epoch: 2 - total_rewards: 16.840, critic_lo

In [9]:
print(replay_buffer[10])

step_number: 10,  states: [[ -6.25175476e-01  -3.83656120e+00  -9.47231293e-01   9.89733815e-01
   -7.74296001e-02  -9.36788600e-03  -1.19766712e-01   5.88203520e-02
    8.09767917e-02  -3.31189007e-01  -1.35536969e+00   2.53667235e-01
   -1.78696021e-01  -9.46197510e-02  -8.89178371e+00   1.54876709e-01
    9.20951068e-01   1.35189489e-01  -1.20881870e-01   3.44906658e-01
   -9.56944346e-01   8.38525414e-01   3.88048559e-01   1.13146555e+00
    3.52377415e+00   1.65761459e+00   7.75623322e+00  -1.00000000e+00
    1.95980072e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   -1.29508138e-01]
 [ -5.81001282e-01  -3.80603123e+00  -1.08864975e+00   9.87806737e-01
   -7.17578232e-02  -1.00077968e-02  -1.37798876e-01   7.43504837e-02
    9.69300643e-02  -3.42407852e-01  -1.41701567e+00   2.69931048e-01
   -2.31277958e-01  -3.25431824e-02  -8.72800446e+00   4.10575867e-02
    9.10638928e-01   1.29475966e-01  -1.22843273e-01   3.72669250e-01
   -1.00400031e+00   9.01509345e-01   3.6019